### Download-Unzip & Detecting & Cropping Faces & Saving

> - Download and Unzip Part File
> - Detection is made on only original video for time saving
> - While cropping fake images original face detections are used

In [1]:
# default_exp face_detection.download_detect_crop_save

In [2]:
#export
from fastai.vision import *

from dfdc.core.core import *
from dfdc.core.download_unzip import *
from dfdc.face_detection.generate_detections import *
from dfdc.face_detection.save_cropped_faces import *

### Download and Unzip

In [3]:
array(download_parts)

array(['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35',
       '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49'], dtype='<U2')

In [3]:
download_part_no = "00"

In [5]:
video_path = download_unzip(download_part_no)

In [13]:
part_no = video_path.name.split("_")[-1]; part_no

'0'

### Detect Faces from Original Videos

In [14]:
modelname = "mobilenet"
data_path = Path("/home/ubuntu/data/dfdc/")
video_path = Path(data_path/f"dfdc_train/dfdc_train_part_{part_no}/")

In [15]:
len(video_path.ls())

1335

In [16]:
metadf = read_metadata(get_files(video_path, extensions=['.json'], recurse=True)[0])

In [20]:
video_files_txt = video_path/"original_video_files.txt"
_ = get_original_video_list(video_path, metadf, video_files_txt)

In [21]:
dest_fname = data_path/f"dfdc_face_detections/part_{part_no}_retina_detections.csv"
freq = 10
model_args = dict(confidence_threshold = 0.5, top_k = 5, nms_threshold = 0.5, keep_top_k = 5)

In [22]:
df = generate_face_detections(video_files_txt, freq, "mobilenet")

  0%|          | 0/86 [00:00<?, ?it/s]

Loading pretrained model from /home/ubuntu/git/dfdc/local_misc/pytorch_retinaface/weights/mobilenet0.25_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300


100%|██████████| 86/86 [01:38<00:00,  1.15s/it]


In [28]:
df.head()

,source,size,face_detections,n_frames,sample_freq,len_video
0,fgobmbcami.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[567, 188, 747...",30,10,300
1,yxvmusxvcz.mp4,"(1920, 1080)","[{'frame_no': 0, 'detections': [[646, 604, 721...",30,10,300
2,prmwoaeeng.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[759, 158, 974...",30,10,300
3,yexeazbqig.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[1461, 224, 16...",30,10,300
4,qyqufaskjs.mp4,"(1920, 1080)","[{'frame_no': 0, 'detections': [[435, 315, 538...",30,10,300


### Crop and Save Faces for All Videos 

In [24]:
metadf["source"] = metadf.apply(lambda o: o['original'] 
                                if type(o['original']) == str else o['fname'], axis=1)

In [25]:
df = df.rename(columns={"fname":"source"})

In [26]:
face_detections_df = metadf.merge(df, how='inner', on='source')

In [27]:
face_detections_df.head()

,fname,label,split,original,source,size,face_detections,n_frames,sample_freq,len_video
0,owxbbpjpch.mp4,FAKE,train,wynotylpnm.mp4,wynotylpnm.mp4,"(1920, 1080)","[{'frame_no': 0, 'detections': [[427, 311, 527...",30,10,300
1,bpguesjrfa.mp4,FAKE,train,wynotylpnm.mp4,wynotylpnm.mp4,"(1920, 1080)","[{'frame_no': 0, 'detections': [[427, 311, 527...",30,10,300
2,skewcclbhg.mp4,FAKE,train,wynotylpnm.mp4,wynotylpnm.mp4,"(1920, 1080)","[{'frame_no': 0, 'detections': [[427, 311, 527...",30,10,300
3,ybdtkypwez.mp4,FAKE,train,wynotylpnm.mp4,wynotylpnm.mp4,"(1920, 1080)","[{'frame_no': 0, 'detections': [[427, 311, 527...",30,10,300
4,qwvxbksoeo.mp4,FAKE,train,wynotylpnm.mp4,wynotylpnm.mp4,"(1920, 1080)","[{'frame_no': 0, 'detections': [[427, 311, 527...",30,10,300


In [29]:
face_detections_csv = data_path/f'dfdc_face_detections/part_{part_no}_retina_detections.csv'

In [30]:
face_detections_df.to_csv(face_detections_csv, index=False)

In [160]:
len(set(face_detections_df.fname))

1334

In [ ]:
save_cropped_faces(data_path, video_path, face_detections_csv)

### Remove videos

In [220]:
shutil.rmtree(video_path)

### Combine All

In [5]:
array(download_parts)

array(['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35',
       '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49'], dtype='<U2')

In [6]:
#export
def download_detect_crop_save(download_part_no):
    # Download and Unzip
    print(f"Downloading part {download_part_no}")
    video_path = download_unzip(download_part_no)
    part_no = video_path.name.split("_")[-1]

    # Detect Faces from Original Videos
    modelname = "mobilenet"
    data_path = Path("/home/ubuntu/data/dfdc/")
    video_path = Path(data_path/f"dfdc_train/dfdc_train_part_{part_no}/")
    metadf = read_metadata(get_files(video_path, extensions=['.json'], recurse=True)[0])
    video_files_txt = video_path/"original_video_files.txt"
    _ = get_original_video_list(video_path, metadf, video_files_txt)

    dest_fname = data_path/f"dfdc_face_detections/part_{part_no}_retina_detections.csv"
    freq = 10
    model_args = dict(confidence_threshold = 0.5, top_k = 5, nms_threshold = 0.5, keep_top_k = 5)
    df = generate_face_detections(video_files_txt, freq, "mobilenet")

    # Crop and Save Faces for All Videos 
    metadf["source"] = metadf.apply(lambda o: o['original'] if type(o['original']) == str else o['fname'], axis=1)
    df = df.rename(columns={"fname":"source"})
    face_detections_df = metadf.merge(df, how='inner', on='source')
    
    # Keep only existing videos (video dir missing videos which is in metadata)
    video_files = get_files(video_path, extensions=['.mp4'], recurse=True)
    video_file_fnames = [o.name for o in video_files]
    face_detections_df = face_detections_df[face_detections_df.fname.isin(video_file_fnames)].reset_index(drop=True)
    
    # Do crop and save
    face_detections_csv = data_path/f'dfdc_face_detections/part_{part_no}_retina_detections.csv'
    face_detections_df.to_csv(face_detections_csv, index=False)
    save_cropped_faces(data_path, video_path, face_detections_csv)

    # Remove videos
    shutil.rmtree(video_path)

### Visualize

In [119]:
crop_path = Path(f'/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfdc_train_part_{part_no}')

In [211]:
len(crop_path.ls())

690

In [ ]:
def show_crop_dir(crop_dir):
    il = ImageList.from_folder(crop_dir)
    n = int(np.ceil(np.sqrt(len(il.items)))); n
    axes = subplots(n,n).flatten()
    for img, ax in zip(il, axes): img.show(ax=ax)

In [ ]:
rand_src = np.random.choice(list(set(metadf.source)))
orig_fname = metadf[metadf.fname == rand_src].fname.values[0]
fnames = metadf[metadf.source == rand_src].fname.values

In [ ]:
crop_dir = (crop_path/Path(orig_fname).stem); print(crop_dir)
show_crop_dir(crop_dir)

In [ ]:
rand_fname = Path(np.random.choice(fnames))
crop_dir = (crop_path/rand_fname.stem); print(crop_dir)
show_crop_dir(crop_dir)

In [215]:
from dfdc.core.video_core import *
play_video(video_path/f'{orig_fname}', video_path/f'{rand_fname}')

### export

In [20]:
from nbdev.export import notebook2script

In [21]:
notebook2script()

Converted 001 - extract_faces.ipynb.
Converted 002 - face_detection_retinaface.ipynb.
Converted 003 - save_face_crops.ipynb.
Converted 004 - tl_baseline.ipynb.
Converted 00_core.ipynb.
Converted 01_video_core.ipynb.
Converted 02_download_unzip_files.ipynb.
Converted 10_bbox_utils.ipynb.
Converted 11_retinaface_detection.ipynb.
Converted 12_generate_face_detections.ipynb.
Converted 13_save_cropped_faces.ipynb.
Converted 14_download_detect_crop_save.ipynb.
Converted 15_extract_all.ipynb.
Converted 20_datasets.ipynb.
Converted 21_single_frame_model.ipynb.
Converted index.ipynb.
Converted inspect original fake pairs for face detection.ipynb.
Converted run_commands.ipynb.
